<a href="https://colab.research.google.com/github/Ashail33/Masters-work/blob/master/Clustering%20data%20creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/Ashail33/mdcgenpy

from  mdcgenpy import clusters as cl
import numpy as np
import pandas as pd

# The properties I will be adding to the dataset are:
#     1) Outliers - Binary ( two options) - outliers
#     2) Noise - Binary ( two options) add_noise=0,n_noise=None,
#     3) Number of clusters - I will select three values k
#     4) Number of data points - I will select three values ( 100 000, 1 000 000, 100 000 000)n_samples
#     5) Number of features - I will select five values ( 2, 10, 50, 100, 500) n_feats
#     6) Density - I will select three values compactness_factor
#     --7) Cluster shape - I will select three types - clusters within a radius ,  hollow shaped clusters , s-shaped / c-shaped clusters
#      distributions
#         'uniform': lambda shape, param: np.random.uniform(-param, param, shape),
#     'gaussian': lambda shape, param: np.random.normal(0, param, shape),
#     'logistic': lambda shape, param: np.random.logistic(0, param, shape),
#     'triangular': lambda shape, param: np.random.triangular(-param, 0, param, shape),
#     'gamma': lambda shape, param: np.random.gamma(2 + 8 * np.random.rand(), param / 5, shape),
#     'gap': lambda shape, param: gap(shape, param)

# --8) Missing values - this will need to be created by randomly removing values up to a certain number of columns and records


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Ashail33/mdcgenpy to /tmp/pip-req-build-nt451c5m
  Running command git clone --filter=blob:none --quiet https://github.com/Ashail33/mdcgenpy /tmp/pip-req-build-nt451c5m
  Resolved https://github.com/Ashail33/mdcgenpy to commit 2e1580719361905a65ca58a42c039a3038276c95
  Preparing metadata (setup.py) ... done
  Created wheel for mdcgenpy: filename=mdcgenpy-1.0.0-py3-none-any.whl size=24119 sha256=62df47454ee43c8f9cc402c847e588ded6ec95bd5e8ece2099a285f3ab8cab5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-9so3cfyl/wheels/17/85/f8/346c4006c2269fb848b0f7f13586b92df1aa0451d93761ec8b
Successfully built mdcgenpy


In [3]:
zz=[[2,1,5],[3,4,3],[6,7,9]]

In [4]:
list(zip(*zz))

[(2, 3, 6), (1, 4, 7), (5, 3, 9)]

In [ ]:
# ,compactness_factor=[0.1,0.2,0.5,0.6,0.01],n_noise=[1,1,1,1,1]

In [ ]:
# ['triangular','triangular','triangular','triangular','triangular']

set

In [5]:
cluster_gen=cl.ClusterGenerator(n_samples=1300000,k=[100000,100000,100000,100000,100000,400000,400000],compactness_factor=[0.1,0.2,0.3,0.4,0.5,0.7,0.1],n_feats=7,distributions=['uniform','triangular','normal','gamma','gap','gamma','normal'],scale=False,outliers=100,rotate=True,mv=[0,1,1,1,0,0,0],)

In [ ]:
# cluster_gen=cl.ClusterGenerator(n_samples=100000,k=5,outliers= 500,n_feats=5,distributions='triangular')

In [ ]:
# data = cluster_gen.generate_data()
# need to figure out where variable shape comes from and change that

In [6]:
# Get tuple with a numpy array with samples and another with labels
data = cluster_gen.generate_data()

y=pd.DataFrame(data[1])# labels
y.columns=['cluster']

x=pd.DataFrame(data[0],) # dataset with features

zz=pd.concat([x,y],axis=1)

99614
100068
99925
99963
99599
399453
401274


In [6]:
%matplotlib inline
import plotly.express as px
import matplotlib.pyplot as plt
df = px.data.iris()
fig = px.scatter_3d(zz, x=0, y=1, z=2,
              color='cluster')
plt.show()

In [9]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(zz, x=0, y=1, z=2,
              color='cluster')
fig.show()

In [ ]:
from mdcgenpy import clusters as cl
import numpy as np
import pandas as pd

# Parameters for generating datasets
outliers = [0, 500]
noise_options = [(0, None), (0.1, 100)]
n_clusters = [3, 5, 10]
n_samples = [100_000, 1_000_000, 100_000_000]
n_feats = [2, 10, 50, 100, 500]
compactness_factors = [0.1, 0.5, 1]

# Distributions for cluster shapes
dist_options = ['uniform', 'gaussian', 'logistic', 'triangular', 'gamma', 'gap']

datasets = []

for outlier in outliers:
    for add_noise, n_noise in noise_options:
        for k in n_clusters:
            for n_sample in n_samples:
                for n_feat in n_feats:
                    for compactness_factor in compactness_factors:
                        for distribution in dist_options:
                            distributions = [distribution] * k
                            cluster_gen = cl.ClusterGenerator(
                                n_samples=n_sample,
                                outliers=outlier,
                                n_feats=n_feat,
                                k=k,
                                distributions=distributions,
                                compactness_factor=compactness_factor,
                                add_noise=add_noise,
                                n_noise=n_noise
                            )
                            data = cluster_gen.generate_data()
                            datasets.append(data)


36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36803
52097
11100
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36776
52121
11103
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36788
51930
11282
36377
52464
11159
36377
5246